In [1]:
from ccnet_spark.pipeline import Pipeline,Config
import time

24/04/07 15:32:46 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.110.96 instead (on interface en0)
24/04/07 15:32:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 15:32:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
config=Config(isSample=True,n_segments=1,sampleRate=0.001,cache_dir="../cache_data/",dump="2019-18")
config

Config(dump='2019-18', cache_dir='../cache_data/', output_dir='../cache_data/', min_len=300, isSample=True, sampleRate=0.001, n_segments=1, pipeline=['real_len', 'hash', 'dedup_keep', 'lid', 'sp', 'lm', 'pp_bucket', 'drop'], spark=<pyspark.sql.session.SparkSession object at 0x11e48f100>)

In [3]:
pipeline=Pipeline(config)
df=pipeline.load_data()
s=time.time()
pipeline.run()
pipeline.save_data()
e=time.time()
print(e-s)

24/04/07 15:32:53 WARN DataSource: All paths were ignored:
  file:/Users/zz/github/cache_data/sdf_parquet/2019-18/_sampleRate_0_segment_0_min_len_300.parquet
2024-04-07 15:32 INFO 5912:root - load segment 0, with sampleRate:0.1%,min_len:300,with date:2019-18


0.8398339748382568


In [4]:
pipeline.df.describe()

DataFrame[summary: string, digest: string, url: string, date_download: string, source_domain: string, cc_segment: string, original_length: string, original_nlines: string, title: string, nlines: string, raw_content: string, length: string, lang: string, score: string, perplexity: string, bucket: string]

In [6]:
from pyspark.sql import functions as F

# 定义聚合函数求和
sum_columns = [
    F.sum('original_length').alias('sum_original_length'),
    F.sum('length').alias('sum_length'),
    F.sum('nlines').alias('sum_nlines'),
    F.sum('original_nlines').alias('sum_original_nlines')
]

# 按照 'bucket' 和 'lang' 字段进行分组，并计算每个组合的数量和求和
aggregated_df = pipeline.df.groupBy('bucket', 'lang').agg(
    F.count('*').alias('count'), *sum_columns
)

# 显示聚合后的结果
aggregated_df.show()


+------+----+-----+-------------------+----------+----------+-------------------+
|bucket|lang|count|sum_original_length|sum_length|sum_nlines|sum_original_nlines|
+------+----+-----+-------------------+----------+----------+-------------------+
|middle|  en|    2|               8413|      8222|       113|                130|
|  tail|  nl|    1|              17259|     16452|       224|                285|
|  tail|  hu|    1|               4798|      4152|        87|                109|
|  tail|  es|    2|              14823|     13461|       214|                302|
|  tail|  de|    2|               7849|      5284|       143|                230|
|  tail|  ru|    3|              19653|     17655|       286|                372|
|  tail|  it|    1|               6663|      6000|        60|                 78|
|  tail|  en|   12|              54707|     41535|      1142|               2020|
|  tail|  ro|    1|               1280|      1128|        44|                 52|
|  tail|  ja|   